# Conformal Inference
Julian Hsu
5jun2022

In this notebook we look at the idea of conformal inference from *Chernozhukov, Wuthrich, and Zhu* : https://arxiv.org/abs/1712.09089. 


In [ ]:

import sys
import os
print (os.getcwd() )


import seaborn as sns

import pandas as pd
import numpy as np
import itertools
import math
import scipy
import functools
import sklearn as sk
from sklearn import impute, pipeline

%matplotlib inline
import matplotlib.pyplot as plt


pd.options.mode.chained_assignment = None  # default='warn'



## Permutations to get P-Values
Implement the $iid$ permutation approach. Given a set of time period, go through different iterations in that time period. 

Our setup is for the number pre-treatment periods $T_0$ and the number of post-treatment periods $T_1$. $\hat{u}_t$ is the difference between the estimated proxy outcome and realized outcome.
$$S_q(\hat{u}) = (T_1^{-1/2}  \sum^T_{t = T_0 + 1}|\hat{u}_t|^q )^{1/q}$$
**Note that $S_q(\hat{u})$ is calculated without any permutations or scrambling of the data.** In order to get p-values, we need to do multiple permutations, each with their own instance of $S_q(\hat{u})$. Denote each of the permutations $\pi \in \Pi$ as $S_q(\hat{u}_{\pi})$. The p-value is then:

$$\hat{p} = 1 - \dfrac{1}{\Pi} \sum_{\pi \in \Pi} 1\{ S_q(\hat{u}_{\pi}) < S_q(\hat{u})  \} $$

We can also calculate the confidence interval by a grid search process. Consider a lot of different treatment effects - for example if you are interested in treatment effects that are constant in all post-treatment periods, it is just one number. Each treatment effect is a "null hypotheses" you incorporate when training the model, so you can calculate $\hat{p}$ by seeing whether it is different from that treatment effect "null hypothesis." Then estimate the confidence interval of $1-\alpha$ (or set, since we are doing a grid search) where those $\hat{p}$ are greater than tha False Positive Rate $\alpha$.


Therefore, our implementation algorithm is:
1. Don't scramble anything and train the model.
    1. Estimate the treatment effect.
    2. Estimate the residual and calculate $S_q({\hat{u})$.
    
2. Pick one way of scrambling the time periods, $\pi$;
3. Estimate the model based on that scrambling, $\pi$.  Calculate $S_q({\hat{u}_\pi)$.
4. Calculate the p-value, $\hat{p}$.
5. If we are interested, calculate the confidence set by 